## select target and vps dataset

In [1]:
import logging
import pickle

from pathlib import Path
from random import choice

from geoloc_imc_2023.cbg import CBG, get_prefix_from_ip
from geoloc_imc_2023.default import (
    ANCHOR_PATH
)

logging.basicConfig(level=logging.INFO)

NB_TARGET = 1 # testing
NB_VP = 1 # testing

ripe_credentials = {
    "username": "timur.friedman@sorbonne-universite.fr",
    "key": "b3d3d4fc-724e-4505-befe-1ad16a70dc87",
}

# load hitlist
hitlist_file = Path(".") / "../datasets/parsed_hitlist.pickle"
with open(hitlist_file, "rb") as f:
    hitlist_addr = pickle.load(f)

# load anchors
anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

# select targets and vps from anchors
targets= [choice(list(anchors)) for _ in range(0, NB_TARGET)]
vps = list(set(anchors).difference(set(targets)))[:NB_VP]

logging.info(f"nb targets: {len(targets)} : {[target for target in targets]}")
logging.info(f"nb_vps : {len(vps)} {[vp for vp in vps]}")

INFO:root:nb targets: 10 : ['37.10.41.14', '103.196.37.98', '178.251.24.49', '207.246.74.246', '103.122.190.141', '192.42.42.166', '152.195.206.35', '37.252.251.26', '188.172.218.22', '63.222.190.5']
INFO:root:nb_vps : 10 ['34.97.153.243', '212.9.170.190', '107.162.219.5', '45.135.151.45', '176.74.17.75', '200.61.6.78', '212.72.227.133', '216.37.47.154', '68.64.60.178', '94.203.76.219']
INFO:root:starting measurement for target_prefix='37.10.41.0' with ['37.10.41.1', '37.10.41.2', '37.10.41.3'] with 10 self.anchors
INFO:root:starting measurement for target_prefix='103.196.37.0' with ['103.196.37.1', '103.196.37.2', '103.196.37.10'] with 10 self.anchors
INFO:root:starting measurement for target_prefix='178.251.24.0' with ['178.251.24.1', '178.251.24.2', '178.251.24.3'] with 10 self.anchors
INFO:root:starting measurement for target_prefix='207.246.74.0' with ['207.246.74.4', '207.246.74.7', '207.246.74.9'] with 10 self.anchors
INFO:root:starting measurement for target_prefix='103.122.190

## Tier 1: probing each target prefixes

In [ ]:
cbg = CBG(targets, vps, ripe_credentials, hitlist_addr, anchors)

# get target prefixes
target_prefixes = []
for target_addr in targets:
    target_prefix = get_prefix_from_ip(target_addr)
    target_prefixes.append(target_prefix)

# initialization: start measurements from all vps to targets
init_measurement_ids = cbg.prefix_probing(target_prefixes)

# retreive measurement results from RIPE Atlas
logging.info(init_measurement_ids)

In [2]:
import pickle

# save results
out_file = f"../results/init_measurement_ids.pickle"
print(out_file)
with open(out_file, "wb") as f:
    pickle.dump(init_measurement_ids,f)

../results/init_measurement_ids.pickle


## Retrieve prefix probing results 

In [1]:
import pickle
from pathlib import Path

from geoloc_imc_2023.query_api import retreive_single_measurement

measurement_ids_file = Path(".") / "../results/init_measurement_ids.pickle"
with open(measurement_ids_file, "rb") as f:
    init_measurement_ids = pickle.load(f)

# load anchors
anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

ripe_credentials = {
    "username": "timur.friedman@sorbonne-universite.fr",
    "key": "b3d3d4fc-724e-4505-befe-1ad16a70dc87",
}

measurement_results = {}
for prefix in init_measurement_ids:
    for measurement_id in init_measurement_ids[prefix]:
        measurement_results[prefix] = retreive_single_measurement(measurement_id, anchors, ripe_credentials["key"])

In [2]:
import pickle

# save results
out_file = f"../results/init_measurement.pickle"
print(out_file)
with open(out_file, "wb") as f:
    pickle.dump(measurement_results,f)

../results/init_measurement.pickle


## Tier 2: probe each target

In [1]:
import logging
import pickle
import asyncio

from pathlib import Path
from random import choice
from timeit import default_timer as timer

from geoloc_imc_2023.cbg import CBG

logging.basicConfig(level=logging.INFO)

NB_TARGET = 3
NB_VP = 1

ripe_credentials = {
    "username": "timur.friedman@sorbonne-universite.fr",
    "key": "b3d3d4fc-724e-4505-befe-1ad16a70dc87",
}

# load anchors
anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

# load hitlist
hitlist_file = Path(".") / "../datasets/parsed_hitlist.pickle"
with open(hitlist_file, "rb") as f:
    hitlist_addr = pickle.load(f)

targets = [anchor_addr for anchor_addr in anchors][:NB_TARGET]
vps = [anchor_addr for anchor_addr in anchors][:NB_VP]

logging.info(f"nb targets: {len(targets)}")
logging.info(f"nb_vps : {len(vps)}")

In [ ]:
cbg = CBG(targets, vps, ripe_credentials, hitlist_addr, anchors)

start = timer()
measurement_ids = await cbg.target_probing(targets, vps)
end =timer()

time_elapsed = end - start
print(time_elapsed)

In [ ]:
import pickle

# save results
out_file = f"../results/target_measurement_ids.pickle"
with open(out_file, "wb") as f:
    pickle.dump(measurement_ids,f)

## Retreive target measurements

In [ ]:
from pathlib import Path
import pickle
from geoloc_imc_2023.query_api import retreive_single_measurement

# load hitlist
in_file = Path(".") / "../results/target_measurement_ids.pickle"
with open(in_file, "rb") as f:
    measurement_ids = pickle.load(f)

# load anchors
anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

ripe_credentials = {
    "username": "timur.friedman@sorbonne-universite.fr",
    "key": "b3d3d4fc-724e-4505-befe-1ad16a70dc87",
}

print(len(measurement_ids))

measurement_results = []
for measurement_id in measurement_ids:
    measurement_result = retreive_single_measurement(measurement_id, anchors, ripe_credentials["key"])
    print(f"got results for target: {measurement_result['target_addr']} from {len(measurement_result['results'])} servers")
    measurement_results.append(measurement_result)

In [15]:
import pickle

# save results
out_file = f"../results/target_measurement.pickle"
print(out_file)
with open(out_file, "wb") as f:
    pickle.dump(measurement_results,f)

../results/target_measurement.pickle
